# Task 03 — Testing ML API End-to-End

Test the FastAPI app from `sample_fastapi_app.py` using TestClient and mocking.

## Setup

In [ ]:
import sys, os

FIXTURES = os.path.abspath(os.path.join("..", "fixtures", "input"))
if not os.path.exists(FIXTURES):
    FIXTURES = os.path.abspath(os.path.join("fixtures", "input"))
sys.path.insert(0, FIXTURES)

from fastapi.testclient import TestClient
from unittest.mock import MagicMock
from sample_fastapi_app import app, ml_models

print("Setup complete.")

## Task 3.1: Test Health Endpoint

Write tests that verify:
- `/health` returns 200
- Response has `status` = "ok" and `model_loaded` = True
- Use `TestClient` with context manager (required for lifespan!)

In [ ]:
# YOUR CODE HERE

# with TestClient(app) as client:
#     ...

# TEST — Do not modify
with TestClient(app) as client:
    resp = client.get("/health")
    assert resp.status_code == 200
    assert resp.json()["status"] == "ok"
    assert resp.json()["model_loaded"] is True
print("Task 3.1 passed!")

## Task 3.2: Test Predict Endpoint

Write tests for `/predict`:
- Positive text returns a valid prediction
- Response has `label`, `score`, and `text` fields
- Score is between 0 and 1

In [ ]:
# YOUR CODE HERE
# Test the /predict endpoint with a positive text

# TEST — Do not modify
with TestClient(app) as client:
    resp = client.post("/predict", json={"text": "This is amazing and great"})
    assert resp.status_code == 200
    data = resp.json()
    assert "label" in data
    assert "score" in data
    assert "text" in data
    assert 0 <= data["score"] <= 1
    assert data["text"] == "This is amazing and great"
print("Task 3.2 passed!")

## Task 3.3: Test Validation Errors

Write tests that verify the API returns 422 for:
- Empty text `{"text": ""}`
- Missing text field `{}`
- Text too long (over 5000 chars)

In [ ]:
# YOUR CODE HERE

# TEST — Do not modify
with TestClient(app) as client:
    assert client.post("/predict", json={"text": ""}).status_code == 422
    assert client.post("/predict", json={}).status_code == 422
    assert client.post("/predict", json={"text": "x" * 5001}).status_code == 422
print("Task 3.3 passed!")

## Task 3.4: Test Batch Predict

Write tests for `/predict/batch`:
- Send 3 texts, get 3 predictions back
- Each prediction has the required fields

In [ ]:
# YOUR CODE HERE

# TEST — Do not modify
with TestClient(app) as client:
    texts = ["good product", "terrible service", "it was okay"]
    resp = client.post("/predict/batch", json={"texts": texts})
    assert resp.status_code == 200
    preds = resp.json()["predictions"]
    assert len(preds) == 3
    for p in preds:
        assert "label" in p and "score" in p and "text" in p
print("Task 3.4 passed!")

## Task 3.5: Mock the ML Model

Replace the sentiment model with a `MagicMock` that always returns `("positive", 0.99)`.
Verify that the mock is used and the response matches.

In [ ]:
# YOUR CODE HERE
# Hint:
# mock_model = MagicMock()
# mock_model.predict.return_value = ("positive", 0.99)
# ml_models["sentiment"] = mock_model
# Then make a request and verify the response

# TEST — Do not modify
with TestClient(app) as client:
    mock_model = MagicMock()
    mock_model.predict.return_value = ("positive", 0.99)
    ml_models["sentiment"] = mock_model

    resp = client.post("/predict", json={"text": "any text here"})
    assert resp.status_code == 200
    assert resp.json()["label"] == "positive"
    assert resp.json()["score"] == 0.99
    mock_model.predict.assert_called_once_with("any text here")
print("Task 3.5 passed!")

## Task 3.6: Test Model Not Loaded (503)

Clear `ml_models` so there's no model, then verify `/predict` returns 503.

In [ ]:
# YOUR CODE HERE
# Hint: manually clear ml_models after lifespan loads
# ml_models.clear()

# TEST — Do not modify
with TestClient(app) as client:
    ml_models.clear()
    resp = client.post("/predict", json={"text": "hello"})
    assert resp.status_code == 503
    assert "not loaded" in resp.json()["detail"].lower()
print("Task 3.6 passed!")